In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os 
import timeit
import math
import time
import sparse
import dask
import dask.array as da
from dask.diagnostics import ProgressBar
from glob import glob
from os import path
import pickle
from tqdm import tqdm

from ll_Balltree import *
%run -i 'll_Balltree.py'

In [2]:
outputDir = 'data/posterior_computation_data/'
gom_masks = xr.open_dataset(outputDir + 'gom_masks.nc')

# GLOBAL CONSTANTS
MIN_LON = np.min(gom_masks['lon'].values)
MAX_LON = np.max(gom_masks['lon'].values)
MIN_LAT = np.min(gom_masks['lat'].values)
MAX_LAT = np.max(gom_masks['lat'].values)

#domain width and height (cell counts)
LAT_SIZE = gom_masks.dims['lat']
LON_SIZE = gom_masks.dims['lon']

#cell size
D_LON = gom_masks["lon"][1].values - gom_masks["lon"][0].values
D_LAT = gom_masks["lat"][1].values - gom_masks["lat"][0].values

BIN_CELL_LATS = gom_masks.bin_cell_lats.values
BIN_CELL_LONS = gom_masks.bin_cell_lons.values

MIN_LON, MAX_LON, MIN_LAT, MAX_LAT,LAT_SIZE,LON_SIZE, D_LON,D_LAT

(-97.98001098632812,
 -76.45999145507812,
 18.140000343322754,
 31.899998664855957,
 345,
 539,
 0.03997802734375,
 0.03999900817871094)

In [3]:
inputDir_Sim = ('data/output_v2/rechunked/')
inputFiles_Sim = sorted(glob(inputDir_Sim + '*.zarr' ))

# Save domain_cell_tree
fileObj = open(outputDir + 'output_dict.obj', 'rb')
output = pickle.load(fileObj)
fileObj.close()

In [4]:
n_cell_beaching = output['n_cell_beaching']
n_cell_source = output['n_cell_source']
n_window_beaching = output['n_window_beaching'] 
n_window_source = output['n_window_source']

particle_count = output['particle_count']

beaching_cells = output['beaching_cells'] 
beaching_cell_tree = output['beaching_cell_tree']

source_cell_mask = output['source_cell_mask']
source_cells = output['source_cells']
source_cell_tree = output['source_cell_tree']

beaching_windows = output['beaching_windows']
source_windows = output['source_windows']
d = output['d']
beaching_ym_mat = output['beaching_ym_mat']
source_ym_mat = output['source_ym_mat']

Compute Trajectories for Posterior Computation

In [7]:
file_particle_counts = []
file_obs_counts = []
file_oob_count = 0
for i in range(5):
        filename = inputFiles_Sim[i]
        df = xr.open_zarr(filename).dropna(dim="trajectory", how="all")
        file_particle_counts.append(df.dims['trajectory'])
        file_obs_counts.append(df.dims['obs'])
        
        oob_raw = df['beached'].values
        file_oob_count += np.shape(np.where(oob_raw[:, -1] == 5))[1]

In [8]:
n_total_particles, n_obs = sum(file_particle_counts), file_obs_counts[0]
(n_total_particles, n_obs, file_oob_count)

(349177, 182, 34761)

In [11]:
print("Out of Bounds Percentage |", 100 * file_oob_count / n_total_particles)

Out of Bounds Precentage: 9.95512304647786
